# サーチエンジンとしての利用手順(検索)


## サーチエンジンとしてのElasticsearch

Elasticsearchは、高速に動作する全文検索エンジンです。  
Apache Lucene ([公式ページ](https://lucene.apache.org/))という全文検索のOSSをクラスタ化することで、大量データの高速な全文検索を実現しています。  
また、多彩なクエリにより、時系列データに対して高度な検索を行うことができます。  
すべてのフィールドに検索用インデックス([05_Indexing ](05_Indexing.ipynb)で説明します)を作成することにより、どのフィールドに対する検索でも高速に動作します。  
  
Elasticsearchに投入された内容は、Kibana ([公式ページ](https://www.elastic.co/jp/products/kibana))という専用の可視化ツールを用いて検索・表示・集計できますが、REST APIを送信することで同様の処理をさせることもできます。  
  
本Notebookでは、REST APIによる基本的な操作方法について説明します。  
本章では、[01_Setup](01_Setup.ipynb#Logstashの設定)で登録したデータを使用します。

## リクエストの基本構文

REST APIのリクエストは、HTTP送信ツール等を利用して送信することができます。  
このNotebookではcurlコマンドを用いて送信しいます。  
例えば、  
「meteorological-data-2015.01 というインデックスの中で、locationがtokyoであるドキュメントを検索する」  
という内容のREST APIの呼び出しは次のようになります。

APIの基本構文と構成要素は次の通りです。

** curl -X<メソッド> 'http://<宛先ホスト>/<処理対象>/<処理の種類>?<処理のオプション>' -d '<条件/投入内容>' **


呼び出しの例と対応させると次のような設定内容になっています。  

| 構成要素 | 例での対応箇所 | 設定する内容 |
|----|----|----|
|メソッド|GET|HTTPでのメソッドです。実施したい処理に応じて次を指定します。<br>・検索:GET<br>・追加:PUT<br>・更新:PUSH<br>・削除:DELETE|
|宛先ホスト|$ES_HOST|ElasticsearchのCoordinate(Client) Nodeのホスト（＋ポート）を指定します。|
|処理対象|meteorological-data-2015.01|検索や編集の対象としたいインデックス、タイプ、ドキュメントを指定します。<br>この例ではインデックスのみを指定してしますが、後ろに<br>～/<インデックス>/<タイプ>/<ドキュメント>/ <br>と続けることで、処理対象をさらに限定することもできます。|
|処理の種類|_search|Elasticsearchに実施させたい処理の内容を指定します。"_search"は検索処理を示します。<br>処理が自明の場合（例えばメソッドにPUTを指定したときは追加処理であるとわかる）<br>は省略されることもあります。|
|処理のオプション|pretty|処理の種類に応じたオプションの指定です。<br>例えば"?pretty"は、応答結果を見やすく整形させるオプションです。|
|条件/投入内容|{"query": {～～|処理の内容や範囲を指定するための条件文です。<br>"-d"はcurlコマンドでのデータ部を指定するオプションですが、ここにJSON形式で条件を指定しています。<br>検索条件を指定する"query"や、集約条件を指定する"aggs"などがあります。<br><br>また追加・更新処理においては、投入したいドキュメント内容をここに記述します。|

----
このあとは、実際の具体的なcurlコマンドを用いて、一通りの操作について説明します。  
**コマンドを実際に発行するために、以下のセルの実行して宛先のホスト、およびインデックスとタイプを変数に保存してください。**  
ES_HOSTには、実際の環境に合わせたCoordinate(Client) Nodeのホストを指定してください。

In [3]:
# 本Notebookで利用する共通的な設定(実行する環境に合わせて書き換えてください)
%env ES_HOST=XXX.XXX.XXX.232:9200
%env INDEX=meteorological-data-*
%env TYPE=logs

env: ES_HOST=XXX.XXX.XXX.232:9200
env: INDEX=meteorological-data-*
env: TYPE=logs


また各種操作のコマンド例は、1章で登録したサンプルデータを対象にして正常な応答が得られるような内容にしてあります。  
未実施の場合、**1章の「[サンプルデータの投入](01_Setup.ipynb#サンプルデータの投入)」を実施してください**。  

サンプルデータのインデックス名、タイプ名は次のようになっています。

| パラメータ | 内容 |
|------------|------|
|インデックス名|meteorological-data-年.月|
|タイプ名|logs|

フィールドの内容については、「[サンプルデータの投入](01_Setup.ipynb#サンプルデータの投入)」を参照してください。

## ドキュメントの検索

### 基本構文

検索操作でのREST APIの基本構文は次のようになります。

前述したように、<処理対象>には  

** /<インデックス>/<タイプ>/<ドキュメント>/**  

のような指定をすることで、検索範囲を限定することができます。
具体的には

- /<インデックス>/
- /<インデックス>/<タイプ>/
- /<インデックス>/<タイプ>/<ドキュメント>/

のような指定が可能です。後側のドキュメント等は省略できますが、前側のインデックスは省略できません。

?pretty は応答内容を整形するためのオプションです。指定は任意ですが、見やすさのためには指定した方がよいでしょう。  
整形しない場合は次のように1行で表示されます。

整形する場合は次のようになります。

本Notebookでは?prettyを指定した形式の例を掲載しています。

条件部にはQuery DSL(Domain Specific Language)と呼ばれる独自のJSON文を指定します。  
内容については後述します。

この後は、まず処理対象のインデックス、タイプ、ドキュメントの指定方法について説明し、  
次に条件部のQuery DSLについて説明し、
最後に件数やソート順の指定方法について説明します。

### レスポンスの基本構文（ドキュメントIDで検索した場合）

[ドキュメントIDで検索](#対象指定:ドキュメントIDで検索)した場合のレスポンスは次のようになります。

レスポンスの例と対応させると次のような設定内容になっています。  

| 構成要素 | 例での対応箇所 | 設定される内容 |
|----|----|----|
|インデックス名|\_index|検索したドキュメントのインデックス名。|
|タイプ名|\_type|検索したドキュメントのタイプ名。|
|ドキュメントID|\_id|検索したドキュメントのドキュメントID。|
|ドキュメントのバージョン|\_version|検索したドキュメントのバージョン。<BR>ドキュメントを更新するとバージョンがインクリメントされます。|
|検索結果の有無|found|ドキュメントが存在する場合はtrue、<BR>存在しない場合はfalseが設定されます。|
|ドキュメントの内容|\_source|ドキュメントの内容。<BR>JSON形式で設定されます。|

詳細は、Elasticsearch Referenceの[Get API](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-get.html)を参照してください。

### レスポンスの基本構文（ドキュメントID以外で検索した場合）

ドキュメントID以外の方法(条件指定等)で検索した場合のレスポンスは次のようになります。

レスポンスの例と対応させると次のような設定内容になっています。  

| 構成要素 | 例での対応箇所 | 設定される内容 |
|----|----|----|
|検索時間|took|Elasticsearch内部で検索に要した時間をミリ秒単位で設定されます。|
|タイムアウトの有無|timed_out|レスポンスがタイムアウトした場合はtrue、<BR>タイムアウトしなかった場合はfalseが設定されます。|
|ヒットしたシャード数|\_shards 配下の total|ドキュメントがヒットしたシャード数が設定されます。|
|検索に成功したシャード数|\_shards 配下の successful|検索に成功したシャード数が設定されます。|
|検索に失敗したシャード数|\_shards 配下の failed|検索に失敗したシャード数が設定されます。|
|全ヒット件数|hits 配下の total|全ヒット件数が設定されます。|
|最大スコア値|hits 配下の max_score|検索結果の最大スコア値が設定されます。|
|ドキュメントの内容|hits 配下の hits|ドキュメントの内容がJSON形式の配列で指定されます。<BR>サイズパラメータで指定された件数まで設定されます。<BR>そのため、全ヒット件数より少ない場合があります。|

詳細は、Elasticsearch Referenceの[The Search API](https://www.elastic.co/guide/en/elasticsearch/reference/current/_the_search_api.html)を参照してください。

### 対象指定:インデックス名で検索

あるインデックス配下を検索する場合、次の形式で指定します。  

インデックス名が「meteorological-data-2015.01」でlocationが「tokyo」のドキュメントを検索するコマンドは次の通りです。

In [55]:
%%bash
curl -XGET "http://$ES_HOST/meteorological-data-2015.01/_search?pretty" -d @- << EOF
{
  "query": {
    "term" : { "location" : "tokyo" } 
  }
}
EOF

{
  "took" : 5,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "hits" : {
    "total" : 29,
    "max_score" : 0.105360515,
    "hits" : [
      {
        "_index" : "meteorological-data-2015.01",
        "_type" : "logs",
        "_id" : "tokyo_15-01-03",
        "_score" : 0.105360515,
        "_source" : {
          "date" : "15-01-03",
          "wind_speed_avg" : "3.0",
          "snowfall" : null,
          "temperature_max" : "8.9",
          "precipitation_day" : null,
          "wind_speed_max" : "7.4",
          "sunshine_duration" : "8.9",
          "temperature_avg" : "3.8",
          "temperature_min" : "-1.1",
          "@version" : "1",
          "wind_direction_max_moment" : "北西",
          "information_night" : "後薄曇",
          "precipitation_max_10min" : null,
          "precipitation_max_hour" : null,
          "wind_direction" : "北西",
          "information_daytime" : "晴",
          "@timestamp" : "2015-01-02T15:

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11438  100 11383  100    55  1058k   5235 --:--:-- --:--:-- --:--:-- 1111k


### 対象指定:複数のインデックス名で検索

複数のインデックス名を指定したい場合、インデックス名をカンマ(,)で区切って列記します。

インデックス名が「meteorological-data-2015.01」「meteorological-data-2015.02」のどちらからにあるドキュメントを検索するコマンドは次の通りです。

In [56]:
%%bash
curl -XGET "http://$ES_HOST/meteorological-data-2015.01,meteorological-data-2015.02/_search?pretty" -d @- << EOF
{
  "query": {
    "term" : { "location" : "tokyo" } 
  }
}
EOF

{
  "took" : 5,
  "timed_out" : false,
  "_shards" : {
    "total" : 10,
    "successful" : 10,
    "failed" : 0
  },
  "hits" : {
    "total" : 57,
    "max_score" : 0.18232156,
    "hits" : [
      {
        "_index" : "meteorological-data-2015.02",
        "_type" : "logs",
        "_id" : "tokyo_15-02-02",
        "_score" : 0.18232156,
        "_source" : {
          "date" : "15-02-02",
          "wind_speed_avg" : "4.7",
          "snowfall" : null,
          "temperature_max" : "9.6",
          "precipitation_day" : null,
          "wind_speed_max" : "9.1",
          "sunshine_duration" : "9.6",
          "temperature_avg" : "4.7",
          "temperature_min" : "1.0",
          "@version" : "1",
          "wind_direction_max_moment" : "西",
          "information_night" : null,
          "precipitation_max_10min" : null,
          "precipitation_max_hour" : null,
          "wind_direction" : "西",
          "information_daytime" : "晴",
          "@timestamp" : "2015-02-01T15:00:0

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11427  100 11372  100    55  1215k   6021 --:--:-- --:--:-- --:--:-- 1233k


### 対象指定:インデックス名の部分一致検索

インデックス名は部分一致で指定することもできます。

インデックス名が「meteorological-data-\*」に一致するものからドキュメントを検索するコマンドは次の通りです。

In [57]:
%%bash
curl -XGET "http://$ES_HOST/meteorological-data-*/_search?pretty" -d @- << EOF
{
  "query": {
    "term" : { "location" : "tokyo" } 
  }
}
EOF

{
  "took" : 11,
  "timed_out" : false,
  "_shards" : {
    "total" : 65,
    "successful" : 65,
    "failed" : 0
  },
  "hits" : {
    "total" : 358,
    "max_score" : 0.2876821,
    "hits" : [
      {
        "_index" : "meteorological-data-2014.12",
        "_type" : "logs",
        "_id" : "tokyo_15-01-01",
        "_score" : 0.2876821,
        "_source" : {
          "date" : "15-01-01",
          "wind_speed_avg" : "4.4",
          "snowfall" : null,
          "temperature_max" : "8.2",
          "precipitation_day" : "0",
          "wind_speed_max" : "9.4",
          "sunshine_duration" : "1.3",
          "temperature_avg" : "3.7",
          "temperature_min" : "0.7",
          "@version" : "1",
          "wind_direction_max_moment" : "北西",
          "information_night" : "晴",
          "precipitation_max_10min" : "0",
          "precipitation_max_hour" : "0",
          "wind_direction" : "西",
          "information_daytime" : "時々晴一時雪",
          "@timestamp" : "2014-12-31T15:00

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11439  100 11384  100    55   760k   3762 --:--:-- --:--:-- --:--:--  794k


### 対象指定:全インデックスの検索

インデックスを絞らず全ドキュメントを検索したい場合は、インデックス名の代わりに"_all"という予約語を指定します。

全インデックスからドキュメントを検索するコマンドは次の通りです。

In [58]:
%%bash
curl -XGET "http://$ES_HOST/_all/_search?pretty" -d @- << EOF
{
  "query": {
    "term" : { "location" : "tokyo" } 
  }
}
EOF

{
  "took" : 5,
  "timed_out" : false,
  "_shards" : {
    "total" : 65,
    "successful" : 65,
    "failed" : 0
  },
  "hits" : {
    "total" : 358,
    "max_score" : 0.2876821,
    "hits" : [
      {
        "_index" : "meteorological-data-2014.12",
        "_type" : "logs",
        "_id" : "tokyo_15-01-01",
        "_score" : 0.2876821,
        "_source" : {
          "date" : "15-01-01",
          "wind_speed_avg" : "4.4",
          "snowfall" : null,
          "temperature_max" : "8.2",
          "precipitation_day" : "0",
          "wind_speed_max" : "9.4",
          "sunshine_duration" : "1.3",
          "temperature_avg" : "3.7",
          "temperature_min" : "0.7",
          "@version" : "1",
          "wind_direction_max_moment" : "北西",
          "information_night" : "晴",
          "precipitation_max_10min" : "0",
          "precipitation_max_hour" : "0",
          "wind_direction" : "西",
          "information_daytime" : "時々晴一時雪",
          "@timestamp" : "2014-12-31T15:00:

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11438  100 11383  100    55  1457k   7213 --:--:-- --:--:-- --:--:-- 1389k


### 対象指定:タイプ名で検索

インデックス名に加えてタイプ名を指定する場合は、インデックス名に続いてタイプ名を指定します。

インデックス名が「meteorological-data-2015.01」、タイプ名が「log」のドキュメントを検索するコマンドは次の通りです。

In [59]:
%%bash
curl -XGET "http://$ES_HOST/meteorological-data-2015.01/logs/_search?pretty" -d @- << EOF
{
  "query": {
    "term" : { "location" : "tokyo" } 
  }
}
EOF

{
  "took" : 5,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "hits" : {
    "total" : 29,
    "max_score" : 0.105360515,
    "hits" : [
      {
        "_index" : "meteorological-data-2015.01",
        "_type" : "logs",
        "_id" : "tokyo_15-01-03",
        "_score" : 0.105360515,
        "_source" : {
          "date" : "15-01-03",
          "wind_speed_avg" : "3.0",
          "snowfall" : null,
          "temperature_max" : "8.9",
          "precipitation_day" : null,
          "wind_speed_max" : "7.4",
          "sunshine_duration" : "8.9",
          "temperature_avg" : "3.8",
          "temperature_min" : "-1.1",
          "@version" : "1",
          "wind_direction_max_moment" : "北西",
          "information_night" : "後薄曇",
          "precipitation_max_10min" : null,
          "precipitation_max_hour" : null,
          "wind_direction" : "北西",
          "information_daytime" : "晴",
          "@timestamp" : "2015-01-02T15:

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11438  100 11383  100    55  1556k   7699 --:--:-- --:--:-- --:--:-- 1852k


### 対象指定:ドキュメントIDで検索

ドキュメントIDを指定する場合、インデックス名、タイプ名、ドキュメントIDを指定します。

※条件も指定できますが、ドキュメントIDを指定した場合は応答がたかだか1件ですので、通常は無意味です。

例えば、インデックス名が「meteorological-data-2015.01」、タイプ名が「logs」でドキュメントIDが「tokyo_15-01-08」のドキュメントを検索するコマンドは次の通りです。

In [135]:
!curl -XGET "http://$ES_HOST/meteorological-data-2015.01/logs/tokyo_15-01-08?pretty"

{
  "_index" : "meteorological-data-2015.01",
  "_type" : "logs",
  "_id" : "tokyo_15-01-08",
  "_version" : 1,
  "found" : true,
  "_source" : {
    "date" : "15-01-08",
    "wind_speed_avg" : "4.4",
    "snowfall" : null,
    "temperature_max" : "11.2",
    "precipitation_day" : null,
    "wind_speed_max" : "8.4",
    "sunshine_duration" : "8.9",
    "temperature_avg" : "6.2",
    "temperature_min" : "2.0",
    "@version" : "1",
    "wind_direction_max_moment" : "北西",
    "information_night" : "晴",
    "precipitation_max_10min" : null,
    "precipitation_max_hour" : null,
    "wind_direction" : "西",
    "information_daytime" : "晴",
    "@timestamp" : "2015-01-07T15:00:00.000Z",
    "humidity_avg" : "33.0",
    "humidity_min" : "20.0",
    "atmospheric_pressure" : "1006.2",
    "location" : "tokyo",
    "wind_speed_max_moment" : "13.6",
    "snowfall_max" : null,
    "sea_level_pressure" : "1009.2"
  }
}


### 条件指定:Query DSLの共通的な説明

Elasticsearchに操作を行う際、HTTPのボディはQuery DSL(Domain Specific Language)と呼ばれる独自言語で記述します。

JSONの第一階層に、size(取得サイズ)、query(条件)といったパラメータを指定します。  
第二階層以下の階層については、第一階層毎に異なるため、個々の条件の説明で記載します。  
パラメータは複数設定することができます。

第一階層に複数のパラメータを設定することで、「条件と取得サイズを指定」のような検索を行うことができます。

詳細は、Elasticsearch Referenceの[Request Body Search](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/search-request-body.html)を参照してください。

### 条件指定:単数値で検索

特定のフィールドの値を指定する場合、"term"に続いてフィールド名と値を記述します。


詳細は、Elasticsearch Referenceの[Term Query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-term-query.html)を参照してください。

locationが「tokyo」のドキュメントを検索するコマンドは次の通りです。

In [61]:
%%bash
curl -XGET "http://$ES_HOST/$INDEX/_search?pretty" -d @- << EOF
{
  "query": {
    "term" : { "location" : "tokyo" } 
  }
}
EOF

{
  "took" : 10,
  "timed_out" : false,
  "_shards" : {
    "total" : 65,
    "successful" : 65,
    "failed" : 0
  },
  "hits" : {
    "total" : 358,
    "max_score" : 0.2876821,
    "hits" : [
      {
        "_index" : "meteorological-data-2014.12",
        "_type" : "logs",
        "_id" : "tokyo_15-01-01",
        "_score" : 0.2876821,
        "_source" : {
          "date" : "15-01-01",
          "wind_speed_avg" : "4.4",
          "snowfall" : null,
          "temperature_max" : "8.2",
          "precipitation_day" : "0",
          "wind_speed_max" : "9.4",
          "sunshine_duration" : "1.3",
          "temperature_avg" : "3.7",
          "temperature_min" : "0.7",
          "@version" : "1",
          "wind_direction_max_moment" : "北西",
          "information_night" : "晴",
          "precipitation_max_10min" : "0",
          "precipitation_max_hour" : "0",
          "wind_direction" : "西",
          "information_daytime" : "時々晴一時雪",
          "@timestamp" : "2014-12-31T15:00

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11439  100 11384  100    55   889k   4398 --:--:-- --:--:-- --:--:--  926k


### 条件指定:複数値で検索

特定のフィールドの複数値を指定する場合、"terms"に続いてフィールド名と複数の値を記述します。

単数値は"term"を指定していたのに対し、複数値では"term**s**"と複数形になっている点に注意してください。
また、検索する値は配列で指定します。
詳細は、Elasticsearch Referenceの[Terms Query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-terms-query.html)を参照してください。

wind_direction（風向き）が「西」と「東」のドキュメントを検索するコマンドは次の通りです。

In [62]:
%%bash
curl -XGET "http://$ES_HOST/$INDEX/_search?pretty" -d @- << EOF
{
  "query": {
    "terms" : {
      "wind_direction": ["西", "東"]
    }
  }
}
EOF

{
  "took" : 7,
  "timed_out" : false,
  "_shards" : {
    "total" : 65,
    "successful" : 65,
    "failed" : 0
  },
  "hits" : {
    "total" : 123,
    "max_score" : 2.0794415,
    "hits" : [
      {
        "_index" : "meteorological-data-2015.10",
        "_type" : "logs",
        "_id" : "tokyo_15-10-24",
        "_score" : 2.0794415,
        "_source" : {
          "date" : "15-10-24",
          "wind_speed_avg" : "2.3",
          "snowfall" : null,
          "temperature_max" : "23.9",
          "precipitation_day" : null,
          "wind_speed_max" : "8.1",
          "sunshine_duration" : "7.6",
          "temperature_avg" : "19.0",
          "temperature_min" : "15.2",
          "@version" : "1",
          "wind_direction_max_moment" : "北西",
          "information_night" : "晴",
          "precipitation_max_10min" : null,
          "precipitation_max_hour" : null,
          "wind_direction" : "西",
          "information_daytime" : "一時曇",
          "@timestamp" : "2015-10-23T15:

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11500  100 11425  100    75  1268k   8524 --:--:-- --:--:-- --:--:-- 1394k


### 条件指定:範囲検索（数値）

特定のフィールドが取る数値の範囲を指定する場合、"range"を用います。

指定可能なパラメータは次の通りです。  
どの項目も指定は任意であり、下限だけ指定するようなクエリも可能です。

| パラメータ名 | 意味 |
|--------------|------|
|gte|「検索対象 ≧ 指定値」という条件。|
|gt|「検索対象 ＞ 指定値」という条件。|
|lte|「検索対象 ≦ 指定値」という条件。|
|lt|「検索対象 ＜ 指定値」という条件。|
|boost|検索結果に対する重みづけ。<br>詳細は、The Definitive Guideの[Query-Time Boosting](https://www.elastic.co/guide/en/elasticsearch/guide/current/query-time-boosting.html)を参照してください。|

詳細は、Elasticsearch Referenceの[Range Query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-range-query.html)を参照してください。

temperature_maxが「30℃以上」かつ「35℃未満」のドキュメントを検索するケース(2015年8月の真夏日を検索するケース)のコマンドは次の通りです。

In [63]:
%%bash
curl -XGET "http://$ES_HOST/meteorological-data-2015.08/_search?pretty" -d @- << EOF
{
  "query": {
    "range" : {
      "temperature_max" : {
        "gte" : 30,
        "lt" : 35
      }
    }
  }
}
EOF

{
  "took" : 3,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "hits" : {
    "total" : 12,
    "max_score" : 1.0,
    "hits" : [
      {
        "_index" : "meteorological-data-2015.08",
        "_type" : "logs",
        "_id" : "tokyo_15-08-09",
        "_score" : 1.0,
        "_source" : {
          "date" : "15-08-09",
          "wind_speed_avg" : "2.5",
          "snowfall" : null,
          "temperature_max" : "33.4",
          "precipitation_day" : null,
          "wind_speed_max" : "4.6",
          "sunshine_duration" : "3.4",
          "temperature_avg" : "27.9",
          "temperature_min" : "24.7",
          "@version" : "1",
          "wind_direction_max_moment" : "南東",
          "information_night" : "後曇",
          "precipitation_max_10min" : null,
          "precipitation_max_hour" : null,
          "wind_direction" : "南東",
          "information_daytime" : "一時曇",
          "@timestamp" : "2015-08-08T15:00:00.000Z",


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11506  100 11399  100   107  2309k  22194 --:--:-- --:--:-- --:--:-- 2782k


### 条件指定:範囲検索（日時）

数値でなく日時の範囲を指定する場合も"range"が使えますが、利用できるパラメータが異なります。

指定可能なパラメータは次の通りです。  
どの項目も指定は任意であり、下限だけ指定するようなクエリも可能です。

| パラメータ名 | 意味 |
|--------------|------|
|gte|「検索対象 ≧ 指定値」という条件。|
|gt|「検索対象 ＞ 指定値」という条件。|
|lte|「検索対象 ≦ 指定値」という条件。|
|lt|「検索対象 ＜ 指定値」という条件。|
|boost|検索結果に対する重みづけ。<br>詳細は、The Definitive Guideの[Query-Time Boosting](https://www.elastic.co/guide/en/elasticsearch/guide/current/query-time-boosting.html)を参照してください。|
|format|下限値、上限値のフォーマット。<br>詳細は、Elasticsearch Referenceの[format](https://www.elastic.co/guide/en/elasticsearch/reference/current/mapping-date-format.html)を参照してください。|
|time_zone|下限値、上限値のタイムゾーン。<br>例えば日本であれば"+9:00"と指定するなどです。<br>また、"Asia/Tokyo"のような名称でも記述することができます。[Joda-Time](http://www.joda.org/joda-time/timezones.html)の「Canonical ID」を名称に利用できます。|

なお、日時の代わりに"**now**"という予約語を設定すると、現在日時という意味になります。  
詳細は、Elasticsearch Referenceの[Date Math](https://www.elastic.co/guide/en/elasticsearch/reference/current/common-options.html#date-math)を参照してください。  
  
クエリの詳細は、Elasticsearch Referenceの[Range Query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-range-query.html)を参照してください。


日本時間で2015/8/2からの１週間のドキュメントを検索するコマンドは次の通りです。

In [64]:
%%bash
curl -XGET "http://$ES_HOST/meteorological-data-2015.08/_search?pretty" -d @- << EOF
{
  "query": {
    "range" : {
      "@timestamp" : {
        "gte": "2015-08-02",
        "lt": "2015-08-09",
        "time_zone": "+09:00"
      }
    }
  }
}
EOF

{
  "took" : 3,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "hits" : {
    "total" : 7,
    "max_score" : 1.0,
    "hits" : [
      {
        "_index" : "meteorological-data-2015.08",
        "_type" : "logs",
        "_id" : "tokyo_15-08-04",
        "_score" : 1.0,
        "_source" : {
          "date" : "15-08-04",
          "wind_speed_avg" : "3.4",
          "snowfall" : null,
          "temperature_max" : "35.1",
          "precipitation_day" : null,
          "wind_speed_max" : "6.6",
          "sunshine_duration" : "11.4",
          "temperature_avg" : "30.0",
          "temperature_min" : "26.5",
          "@version" : "1",
          "wind_direction_max_moment" : "南東",
          "information_night" : null,
          "precipitation_max_10min" : null,
          "precipitation_max_hour" : null,
          "wind_direction" : "南東",
          "information_daytime" : null,
          "@timestamp" : "2015-08-03T15:00:00.000Z",
 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8129  100  7979  100   150  1329k  25601 --:--:-- --:--:-- --:--:-- 1558k


### 条件指定:AND条件で検索

複数の条件のすべてに一致するドキュメントを検索する場合、"bool"の下に"must"を指定し、その中に条件を列挙します。

詳細は、Elasticsearch Referenceの[Bool Query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-bool-query.html)を参照してください。

例えば、

- locationが「東京」
- temperature_maxが「30℃以上」かつ「35℃未満」

という条件をすべて満たすドキュメントを検索するケース(東京の真夏日を検索するケース)のコマンドは次の通りです。

In [65]:
%%bash
curl -XGET "http://$ES_HOST/$INDEX/_search?pretty" -d @- << EOF
{
  "query": {
    "bool": {
      "must": [
        {
          "term": {
            "location": "tokyo"
          }
        },
        {
          "range": {
            "temperature_max" : {
              "gte" : 30,
              "lt" : 35
            }
          }
        }
      ]
    }
  }
}
EOF

{
  "took" : 6,
  "timed_out" : false,
  "_shards" : {
    "total" : 65,
    "successful" : 65,
    "failed" : 0
  },
  "hits" : {
    "total" : 36,
    "max_score" : 1.1053605,
    "hits" : [
      {
        "_index" : "meteorological-data-2015.08",
        "_type" : "logs",
        "_id" : "tokyo_15-08-09",
        "_score" : 1.1053605,
        "_source" : {
          "date" : "15-08-09",
          "wind_speed_avg" : "2.5",
          "snowfall" : null,
          "temperature_max" : "33.4",
          "precipitation_day" : null,
          "wind_speed_max" : "4.6",
          "sunshine_duration" : "3.4",
          "temperature_avg" : "27.9",
          "temperature_min" : "24.7",
          "@version" : "1",
          "wind_direction_max_moment" : "南東",
          "information_night" : "後曇",
          "precipitation_max_10min" : null,
          "precipitation_max_hour" : null,
          "wind_direction" : "南東",
          "information_daytime" : "一時曇",
          "@timestamp" : "2015-08-08T15

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11763  100 11483  100   280  1332k  33273 --:--:-- --:--:-- --:--:-- 1401k


### 条件指定:OR条件で検索

複数の条件のいずれか1つ以上に一致するドキュメントを検索する場合、"bool"の下に"should"を指定し、その中に条件を列挙します。

詳細は、Elasticsearch Referenceの[Bool Query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-bool-query.html)を参照してください。

例えば、

- locationが「東京」
- temperature_maxが「30℃以上」かつ「35℃未満」

という条件のいずれか1つ以上を満たすドキュメントを検索するケース(東京の真夏日を検索するケース)のコマンドは次の通りです。

In [66]:
%%bash
curl -XGET "http://$ES_HOST/$INDEX/_search?pretty" -d @- << EOF
{
  "query": {
    "bool": {
      "should": [
        {
          "term": {
            "location": "tokyo"
          }
        },
        {
          "range": {
            "temperature_max" : {
              "gte" : 30,
              "lt" : 35
            }
          }
        }
      ]
    }
  }
}
EOF

{
  "took" : 9,
  "timed_out" : false,
  "_shards" : {
    "total" : 65,
    "successful" : 65,
    "failed" : 0
  },
  "hits" : {
    "total" : 358,
    "max_score" : 1.1053605,
    "hits" : [
      {
        "_index" : "meteorological-data-2015.08",
        "_type" : "logs",
        "_id" : "tokyo_15-08-09",
        "_score" : 1.1053605,
        "_source" : {
          "date" : "15-08-09",
          "wind_speed_avg" : "2.5",
          "snowfall" : null,
          "temperature_max" : "33.4",
          "precipitation_day" : null,
          "wind_speed_max" : "4.6",
          "sunshine_duration" : "3.4",
          "temperature_avg" : "27.9",
          "temperature_min" : "24.7",
          "@version" : "1",
          "wind_direction_max_moment" : "南東",
          "information_night" : "後曇",
          "precipitation_max_10min" : null,
          "precipitation_max_hour" : null,
          "wind_direction" : "南東",
          "information_daytime" : "一時曇",
          "@timestamp" : "2015-08-08T1

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11766  100 11484  100   282   714k  17970 --:--:-- --:--:-- --:--:--  747k


実は「should」は「複数条件のうち、指定数以上に一致する」ドキュメントを検索することができます。

例えば、3つの条件のうち、2つ以上に一致するドキュメントを検索する場合、次の形式でリクエストを送信します。

一致する最低数を「minimum_should_match」で指定します。  
    「minimum_should_match」のデフォルト値が1であるため、省略すると「複数の条件のいずれか1つ以上に一致」という検索(OR検索)になります。

### 検索結果のソート

ソートした検索結果を取得したい場合、"query"と同じ階層に"sort"を指定することで、ソート条件を指定できます。  

優先度の高い順にフィールド名と、ソート順(昇順・降順)を配列で指定します。  
昇順の場合は「asc」、降順の場合は「desc」を指定します。

詳細は、Elasticsearch Referenceの[Sort](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-request-sort.html)を参照してください。

locationが「tokyo」のドキュメントを、最高気温の高い順（降順）で検索するコマンドは次の通りです。

In [67]:
%%bash
curl -XGET "http://$ES_HOST/$INDEX/_search?pretty" -d @- << EOF
{
  "sort": [
    { "temperature_max" : "desc" }
  ],
  "query": {
    "term" : { "location" : "tokyo" } 
  }
}
EOF

{
  "took" : 6,
  "timed_out" : false,
  "_shards" : {
    "total" : 65,
    "successful" : 65,
    "failed" : 0
  },
  "hits" : {
    "total" : 358,
    "max_score" : null,
    "hits" : [
      {
        "_index" : "meteorological-data-2015.08",
        "_type" : "logs",
        "_id" : "tokyo_15-08-07",
        "_score" : null,
        "_source" : {
          "date" : "15-08-07",
          "wind_speed_avg" : "2.6",
          "snowfall" : null,
          "temperature_max" : "37.7",
          "precipitation_day" : null,
          "wind_speed_max" : "6.1",
          "sunshine_duration" : "9.2",
          "temperature_avg" : "31.5",
          "temperature_min" : "26.8",
          "@version" : "1",
          "wind_direction_max_moment" : "南東",
          "information_night" : "時々晴",
          "precipitation_max_10min" : null,
          "precipitation_max_hour" : null,
          "wind_direction" : "東",
          "information_daytime" : "後時々薄曇",
          "@timestamp" : "2015-08-06T15:00:00.

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11870  100 11766  100   104  1408k  12748 --:--:-- --:--:-- --:--:-- 1436k


### 検索結果の件数指定

検索結果の一部を取得したい場合、オフセットと取得件数を指定して検索結果を取得することができます。  
この場合、次の形式でリクエストを送信します。

オフセットを省略した場合は、最初の検索結果(オフセット=0)から取得することになります。  
取得件数を省略した場合は、最大10件取得することになります。

ただし、大量に検索結果があった場合に少しずつ取得したい、というケースでの利用は注意してください。  
例えば、1回目検索で最初の何件かを取得し、2回目の検索で残りを取得する、というようなケースを考えます。  
毎回再検索を行うため、1回目と2回目の検索の間にデータ更新が行われると、1回目と2回目の検索結果の整合性が取れなくなる可能性があります。  
同じ検索結果を何件かずつ取得する場合は[大量データ取得時の検索](#大量データ取得時の検索)を参照してください。  

詳細は、Elasticsearch Referenceの[From / Size](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/search-request-from-size.html)を参照してください。

locationが「tokyo」のドキュメントで、最高気温のTOP10を検索するコマンドは次の通りです。

In [78]:

%%bash
curl -XGET "http://$ES_HOST/$INDEX/_search?pretty" -d @- << EOF
{
  "from" : 0,
  "size" : 10,
  "sort": [
    { "temperature_max" : "desc" }
  ],
  "query": {
    "term" : { "location" : "tokyo" } 
  }
}
EOF

{
  "took" : 20,
  "timed_out" : false,
  "_shards" : {
    "total" : 65,
    "successful" : 65,
    "failed" : 0
  },
  "hits" : {
    "total" : 358,
    "max_score" : null,
    "hits" : [
      {
        "_index" : "meteorological-data-2015.08",
        "_type" : "logs",
        "_id" : "tokyo_15-08-07",
        "_score" : null,
        "_source" : {
          "date" : "15-08-07",
          "wind_speed_avg" : "2.6",
          "snowfall" : null,
          "temperature_max" : "37.7",
          "precipitation_day" : null,
          "wind_speed_max" : "6.1",
          "sunshine_duration" : "9.2",
          "temperature_avg" : "31.5",
          "temperature_min" : "26.8",
          "@version" : "1",
          "wind_direction_max_moment" : "南東",
          "information_night" : "時々晴",
          "precipitation_max_10min" : null,
          "precipitation_max_hour" : null,
          "wind_direction" : "東",
          "information_daytime" : "後時々薄曇",
          "@timestamp" : "2015-08-06T15:00:00

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11898  100 11767  100   131   433k   4947 --:--:-- --:--:-- --:--:--  441k


## ドキュメントの追加

ドキュメントを追加する場合、次の形式でリクエストを送信します。

詳細は、Elasticsearch Referenceの[Index API](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-index_.html)を参照してください。

例えば、ドキュメントIDが1の次のドキュメントを追加するケースを考えます。

| フィールド | 内容 | 
|------------|------|
|@timestamp|"2015-01-01T00:00:00"|
|location|"tokyo"|
|temperature_avg|3.7|

この場合、次のコマンドを実行します。

In [105]:
%%bash
curl -XPUT "http://$ES_HOST/meteorological-data-2015.01/$TYPE/1?pretty" -d @- << EOF
{
  "@timestamp": "2015-01-01T00:00:00",
  "location": "tokyo",
  "temperature_avg": 3.7
}
EOF

{
  "_index" : "meteorological-data-2015.01",
  "_type" : "logs",
  "_id" : "1",
  "_version" : 4,
  "result" : "created",
  "_shards" : {
    "total" : 2,
    "successful" : 2,
    "failed" : 0
  },
  "created" : true
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   307  100   221  100    86  12187   4742 --:--:-- --:--:-- --:--:-- 13000


応答に  
"created" : true  
と出力されれば、実際に追加が成功しています。

本当に追加できているか、検索して確認してみます。

In [106]:
!curl -XGET "http://$ES_HOST/meteorological-data-2015.01/$TYPE/1?pretty"

{
  "_index" : "meteorological-data-2015.01",
  "_type" : "logs",
  "_id" : "1",
  "_version" : 4,
  "found" : true,
  "_source" : {
    "@timestamp" : "2015-01-01T00:00:00",
    "location" : "tokyo",
    "temperature_avg" : 3.7
  }
}


## ドキュメントの更新

### ドキュメントIDで更新

具体的なドキュメントIDを指定してドキュメントを更新する場合、次の形式でリクエストを送信します。

詳細は、Elasticsearch Referenceの[Update API](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-update.html)を参照してください。

例えば、ドキュメントIDが1のドキュメントを次の内容に更新するケースを考えます。

| フィールド | 内容 | 
|------------|------|
|@timestamp|"2015-01-09T00:00:00"|
|location|"tokyo"|
|temperature_avg|5.0|

この場合、次のコマンドを実行します。

In [9]:
%%bash
curl -XPOST "http://$ES_HOST/meteorological-data-2015.01/$TYPE/tokyo_15-01-09/_update?pretty" -d @- << EOF
{
  "doc": {
    "@timestamp": "2015-01-09T00:00:00",
    "location": "tokyo",
    "temperature_avg": 5.0
  }
}
EOF

{
  "_index" : "meteorological-data-2015.01",
  "_type" : "logs",
  "_id" : "tokyo_15-01-09",
  "_version" : 2,
  "result" : "updated",
  "_shards" : {
    "total" : 2,
    "successful" : 2,
    "failed" : 0
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   319  100   214  100   105   8342   4093 --:--:-- --:--:-- --:--:--  8560


### 条件指定して更新

特定の条件に一致したドキュメントを更新する場合、次の形式でリクエストを送信します。

条件指定を行う場合は、タイプ名、ドキュメントIDを指定する必要はありません。  

詳細は、Elasticsearch Referenceの[Update By Query API](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-update-by-query.html)を参照してください。

例えば、locationがtokyoであるドキュメントを次の内容に更新するケースを考えます。

| フィールド | 内容 | 
|------------|------|
|location|"shinjuku"|

この場合、次のコマンドを実行します。

In [50]:
%%bash
curl -XPOST "http://$ES_HOST/meteorological-data-*/_update_by_query?conflicts=proceed&pretty" -d @- << EOF
{
  "query": {
    "term": {
      "location": "tokyo"
    }
  },
  "script": {
    "inline": "ctx._source.location = 'shinjuku'"
  }
}
EOF

{
  "took" : 98,
  "timed_out" : false,
  "total" : 329,
  "updated" : 329,
  "deleted" : 0,
  "batches" : 1,
  "version_conflicts" : 0,
  "noops" : 0,
  "retries" : {
    "bulk" : 0,
    "search" : 0
  },
  "throttled_millis" : 0,
  "requests_per_second" : -1.0,
  "throttled_until_millis" : 0,
  "failures" : [ ]
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   443  100   317  100   126   3110   1236 --:--:-- --:--:-- --:--:--  3138


## ドキュメントの削除

### ドキュメントIDで削除

具体的なドキュメントIDを指定してドキュメントを削除する場合、次の形式でリクエストを送信します。

詳細は、Elasticsearch Referenceの[Delete API](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-delete.html)を参照してください。

例えば、ドキュメントIDが1のドキュメントを削除する場合、次のコマンドを実行します。

In [107]:
!curl -XDELETE "http://$ES_HOST/meteorological-data-2015.01/$TYPE/1?pretty"

{
  "found" : true,
  "_index" : "meteorological-data-2015.01",
  "_type" : "logs",
  "_id" : "1",
  "_version" : 5,
  "result" : "deleted",
  "_shards" : {
    "total" : 2,
    "successful" : 2,
    "failed" : 0
  }
}


### 条件指定して削除

特定の条件に一致したドキュメントを削除する場合、次の形式でリクエストを送信します。

条件指定を行う場合は、タイプ名、ドキュメントIDを指定する必要はありません。  
削除処理ですが、POSTメソッドであることに注意してください。  

詳細は、Elasticsearch Referenceの[Delete By Query API](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-delete-by-query.html)を参照してください。

例えば、locationがtokyoであるドキュメントを削除する場合、次のコマンドを実行します。

In [6]:
%%bash
curl -POST "http://$ES_HOST/meteorological-data-2015.01/_delete_by_query?conflicts=proceed&pretty" -d @- << EOF
{
  "query": {
    "term": {
      "location": "tokyo"
    }
  }
}
EOF

{
  "took" : 27,
  "timed_out" : false,
  "total" : 29,
  "deleted" : 29,
  "batches" : 1,
  "version_conflicts" : 0,
  "noops" : 0,
  "retries" : {
    "bulk" : 0,
    "search" : 0
  },
  "throttled_millis" : 0,
  "requests_per_second" : -1.0,
  "throttled_until_millis" : 0,
  "failures" : [ ]
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   358  100   298  100    60   8178   1646 --:--:-- --:--:-- --:--:--  8277


## 大量データ取得時の検索

大量データ検索による性能への負荷を防ぐため、デフォルトでは検索結果を最大10件までしか返しません。  
また、sizeパラメータを利用しても最大10000件までしか返すことができません。  
ただし、実際には、10000件以上のデータを返したいケースもあります。  
このようなケースでは、Scroll APIを利用して、何回かに分けて検索結果を取得します。  

Scroll APIを利用するには、検索結果保持期間を指定してリクエストを送信します。

すると、「_scroll_id」というフィールドを含んだレスポンスにが返ってきます。  
この値を「スクロールID」と呼びます。  
2回目以降は以下のリクエストを送信します。

Scroll APIを利用する場合は、次の点に注意してください。

- HTTPメソッドはGETでもPOSTでも構いません。
- 大量データを返す処理であるため、通常の検索処理に比べると時間がかかる可能性があります。
- 1回目の検索時に結果をElasticsearch内部に保持し、この結果を順次返します。そのため、大量の検索結果が存在する場合は、Elasticsearchに負荷がかかります。
- 検索結果保持期間を過ぎた場合、Elasticsearch内部で検索結果は破棄されます。破棄後に検索すると「search_context_missing_exception」というエラー応答が返ってきます。
- 1回目の検索時に結果をElasticsearch内部に保持するため、その後データが更新されてもScroll APIのレスポンスは変化しません。
- 2回目以降のリクエストURLに、インデックス名やタイプ名は記載しないでください。
- 2回目以降のリクエストの際に検索結果保持期間は更新されます。
- すべての検索結果を取得すると、以降の検索結果が0件になります。これにより、検索結果が残っていないと判断できます。

詳細は、Elasticsearch Referenceの[Scroll](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/search-request-scroll.html)を参照してください。

検索結果保持期間を1分にして、東京のドキュメントを10件ずつ検索するコマンドは次の通りです。

In [125]:
%%bash
curl -XGET "http://$ES_HOST/meteorological-data-2015.08/_search?pretty&scroll=1m" -d @- << EOF
{
  "size": 10,
  "query": {
    "term" : { "location" : "tokyo" }
  }
}
EOF

{
  "_scroll_id" : "DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAAVUFm1CSmNHNFNaUkY2QmlWZnRrdHRYa1EAAAAAAAAFVRZtQkpjRzRTWlJGNkJpVmZ0a3R0WGtRAAAAAAAABT4WRHBpak9TdmxTdEs1OHhPZmp2bFdoZwAAAAAAAAVTFm1CSmNHNFNaUkY2QmlWZnRrdHRYa1EAAAAAAAAFVhZtQkpjRzRTWlJGNkJpVmZ0a3R0WGtR",
  "took" : 5,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "hits" : {
    "total" : 31,
    "max_score" : 0.105360515,
    "hits" : [
      {
        "_index" : "meteorological-data-2015.08",
        "_type" : "logs",
        "_id" : "tokyo_15-08-09",
        "_score" : 0.105360515,
        "_source" : {
          "date" : "15-08-09",
          "wind_speed_avg" : "2.5",
          "snowfall" : null,
          "temperature_max" : "33.4",
          "precipitation_day" : null,
          "wind_speed_max" : "4.6",
          "sunshine_duration" : "3.4",
          "temperature_avg" : "27.9",
          "temperature_min" : "24.7",
          "@version" : "1",
          "wind_direction_max_momen

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11796  100 11729  100    67  1286k   7523 --:--:-- --:--:-- --:--:-- 1431k


2回目以降の検索を行うコマンドは次の通りです。(scroll_idパラメータに渡す<スクロールID>の部分は、上の1回目の応答で出力された"_scroll_id"パラメータの値と置き換えて実行してください)

In [126]:
%%bash
curl -XGET "http://$ES_HOST/_search/scroll?pretty" -d @- << EOF
{
  "scroll" : "1m",
  "scroll_id" : "<スクロールID>"
}
EOF

{
  "_scroll_id" : "DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAAVUFm1CSmNHNFNaUkY2QmlWZnRrdHRYa1EAAAAAAAAFVRZtQkpjRzRTWlJGNkJpVmZ0a3R0WGtRAAAAAAAABT4WRHBpak9TdmxTdEs1OHhPZmp2bFdoZwAAAAAAAAVTFm1CSmNHNFNaUkY2QmlWZnRrdHRYa1EAAAAAAAAFVhZtQkpjRzRTWlJGNkJpVmZ0a3R0WGtR",
  "took" : 5,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "hits" : {
    "total" : 31,
    "max_score" : 0.105360515,
    "hits" : [
      {
        "_index" : "meteorological-data-2015.08",
        "_type" : "logs",
        "_id" : "tokyo_15-08-19",
        "_score" : 0.06453852,
        "_source" : {
          "date" : "15-08-19",
          "wind_speed_avg" : "3.0",
          "snowfall" : null,
          "temperature_max" : "31.4",
          "precipitation_day" : "0",
          "wind_speed_max" : "6.0",
          "sunshine_duration" : "5.0",
          "temperature_avg" : "27.8",
          "temperature_min" : "25.7",
          "@version" : "1",
          "wind_direction_max_moment"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11965  100 11699  100   266  1441k  33573 --:--:-- --:--:-- --:--:-- 1632k


すべての検索結果を取得すると、以降の検索結果が0件になります。(次の例では、60秒スリープすることで、タイムアウトさせています)

In [4]:
%%bash
sleep 60
curl -XGET "http://$ES_HOST/_search/scroll?pretty" -d @- << EOF
{
  "scroll" : "1m",
  "scroll_id" : "<スクロールID>"
}
EOF

{
  "error" : {
    "root_cause" : [
      {
        "type" : "search_context_missing_exception",
        "reason" : "No search context found for id [1364]"
      },
      {
        "type" : "search_context_missing_exception",
        "reason" : "No search context found for id [1365]"
      },
      {
        "type" : "search_context_missing_exception",
        "reason" : "No search context found for id [1342]"
      },
      {
        "type" : "search_context_missing_exception",
        "reason" : "No search context found for id [1363]"
      },
      {
        "type" : "search_context_missing_exception",
        "reason" : "No search context found for id [1366]"
      }
    ],
    "type" : "search_phase_execution_exception",
    "reason" : "all shards failed",
    "phase" : "query",
    "grouped" : true,
    "failed_shards" : [
      {
        "shard" : -1,
        "index" : null,
        "reason" : {
          "type" : "search_context_missing_exception",
          "reason" : "No sea

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2322  100  2056  100   266   497k  65939 --:--:-- --:--:-- --:--:--  669k


検索結果保持期間を過ぎた場合、「search_context_missing_exception」というエラー応答が返ってきます。

## 実行クエリ解析

クエリを解析するためにProfile APIを使用します。  

Profile APIは、実行クエリについて詳細情報を提供します。  
クエリがどのように実行されるかをユーザーに示し、特定のクエリが遅い理由を理解し、改善するために使用します。

例として、Profile APIのコマンドは次の通りです。

In [12]:
%%bash
curl -XGET "http://$ES_HOST/meteorological-data-2015.01/_search?pretty" -d @- << EOF
{
  "profile": true,
  "query": {
    "term" : { "location" : "tokyo" } 
  }
}
EOF

{
  "took" : 6,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "hits" : {
    "total" : 0,
    "max_score" : null,
    "hits" : [ ]
  },
  "profile" : {
    "shards" : [
      {
        "id" : "[sjROHXMsTxKfXSdoCbpRSQ][meteorological-data-2015.01][4]",
        "searches" : [
          {
            "query" : [
              {
                "type" : "TermQuery",
                "description" : "location:tokyo",
                "time" : "0.1558500000ms",
                "breakdown" : {
                  "score" : 0,
                  "build_scorer_count" : 0,
                  "match_count" : 0,
                  "create_weight" : 155849,
                  "next_doc" : 0,
                  "match" : 0,
                  "create_weight_count" : 1,
                  "next_doc_count" : 0,
                  "score_count" : 0,
                  "build_scorer" : 0,
                  "advance" : 0,
                  "advance_count" : 0
  

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5995  100  5922  100    73   527k   6659 --:--:-- --:--:-- --:--:--  578k


shard毎に実行したクエリの処理時間を見ることができます。

Profile APIの詳細は、Elasticsearch Referenceの[Profile API](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/search-profile.html)を参照してください。